In [1]:
#Import relevant libraries
import io
import itertools
import sklearn.metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# import datetime
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tensorboard.plugins.hparams import api as hp

In [2]:
#Dataset encoding explaination.
#image labelling - 0=glasses/sunglasses. 1= trousers/jeans. 3= shoes

In [3]:
#Import the datasets and preprocess

data_train = np.load(r'Full Dataset/Primary categories - Train.npz')
data_validation = np.load(r'Full Dataset/Primary categories - Validation.npz')
data_test = np.load(r'Full Dataset/Primary categories - Test.npz')


#t extract the arrays from dataset into input(images) and target(labels)
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']


images_test = data_test['images']
labels_test = data_test['labels']

#Pixel-wise normalization of the training, validation and testing data
images_train = images_train/255.0
images_val  = images_val/255.0
images_test = images_test/255.0







In [4]:
#Define the hyperparameters

BATCH_SIZE = 64
EPOCHS = 15

#Define the hyperparamets to tune and the variations we want to test.
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_FILTER_NUM = hp.HParam('filter_num', hp.Discrete([32, 64, 96, 128]))

METRIC_ACCURACY = 'accuracy'

#Log the hyperparameter with the file writer
with tf.summary.create_file_writer('Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM], 
        metrics= [hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
    )



In [5]:
def train_model(hparams,session_num):
    
    
#Create and train the model


    
    model = Sequential([
                Conv2D(32, 3, activation= 'relu', input_shape=(120,90,3)),
                MaxPooling2D(pool_size=(2,2)),
                Conv2D(32, 3, activation= 'relu'),
                MaxPooling2D(pool_size=(2,2)),
                Flatten(),
                Dense(3)
    ])


#describe the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#Compile the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    log_dir = f"Logs/Model 1/fit/run-{session_num}"

    
        
        
        
#Plot_to_image

    def plot_confusion_matrix(cm, class_names):
 
        figure = plt.figure(figsize=(8, 8))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Compute the labels from the normalized confusion matrix.
        labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    



    def plot_to_image(figure):
       
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image


        
    # log the confusion matrix as a heatmap
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')


    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        val_pred_raw = model.predict(images_val)
        val_pred = np.argmax(val_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = confusion_matrix(labels_val, val_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["Glasses/Sunglassses", "Trousers/Jeans", "Shoes"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion_matrix", cm_image, step=epoch)

    # Define the per-epoch callback.
    cm_callback = LambdaCallback(on_epoch_end=log_confusion_matrix)


    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


    early_stopping = EarlyStopping(
        monitor= 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0,
        restore_best_weights = True
    )


#Train the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val, labels_val),
        verbose =2
    )

    _,accuracy = model.evaluate(images_val, labels_val)

    model.save(f"saved_model/Model 1/fit/run-{session_num}")


    return accuracy



In [6]:
#Log the hyperparameter to the TensorBoard



def run(log_dir, hparams,session_num):
    hyperparameter_writer = tf.summary.create_file_writer(log_dir)
    
    
    with hyperparameter_writer.as_default():
        hp.hparams(hparams) #record the values used in this trial
        accuracy = train_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
        
        
        
        
#Train the model with the different hyperparameters
session_num = 1
for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        hparams ={
        HP_FILTER_SIZE: filter_size,
        HP_FILTER_NUM : filter_num
        }
        
        run_name = "run-%d" % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)
        
        session_num += 1
       
        

       
        
        
        








---Starting trial: run-1
{'filter_size': 3, 'filter_num': 32}
Epoch 1/15
51/51 [==============================] - 2s 30ms/step
203/203 - 39s - loss: 0.0965 - accuracy: 0.9724 - val_loss: 0.0066 - val_accuracy: 0.9994 - 39s/epoch - 194ms/step
Epoch 2/15
51/51 [==============================] - 2s 29ms/step
203/203 - 40s - loss: 0.0112 - accuracy: 0.9982 - val_loss: 0.0034 - val_accuracy: 0.9994 - 40s/epoch - 198ms/step
Epoch 3/15
51/51 [==============================] - 1s 29ms/step
203/203 - 40s - loss: 0.0121 - accuracy: 0.9983 - val_loss: 0.0190 - val_accuracy: 0.9963 - 40s/epoch - 199ms/step
Epoch 4/15
51/51 [==============================] - 2s 29ms/step
203/203 - 41s - loss: 0.0070 - accuracy: 0.9992 - val_loss: 0.0033 - val_accuracy: 0.9994 - 41s/epoch - 201ms/step
Epoch 5/15
51/51 [==============================] - 2s 30ms/step
203/203 - 41s - loss: 0.0065 - accuracy: 0.9991 - val_loss: 0.0048 - val_accuracy: 0.9981 - 41s/epoch - 200ms/step
Epoch 6/15
51/51 [====================

INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-1\assets


---Starting trial: run-2
{'filter_size': 3, 'filter_num': 64}
Epoch 1/15
51/51 [==============================] - 2s 36ms/step
203/203 - 52s - loss: 0.1115 - accuracy: 0.9674 - val_loss: 0.0110 - val_accuracy: 0.9975 - 52s/epoch - 257ms/step
Epoch 2/15
51/51 [==============================] - 2s 33ms/step
203/203 - 49s - loss: 0.0143 - accuracy: 0.9985 - val_loss: 0.0042 - val_accuracy: 0.9994 - 49s/epoch - 242ms/step
Epoch 3/15
51/51 [==============================] - 2s 31ms/step
203/203 - 48s - loss: 0.0102 - accuracy: 0.9986 - val_loss: 0.0059 - val_accuracy: 0.9988 - 48s/epoch - 235ms/step
Epoch 4/15
51/51 [==============================] - 2s 31ms/step
203/203 - 46s - loss: 0.0069 - accuracy: 0.9993 - val_loss: 0.0029 - val_accuracy: 0.9994 - 46s/epoch - 228ms/step
Epoch 5/15
51/51 [==============================] - 2s 30ms/step
203/203 - 45s - loss: 0.0061 - accuracy: 0.9994 - val_loss: 0.0037 - val_accuracy: 0.9994 - 45s/epoch - 222ms/step
Epoch 6/15
51/51 [====================

INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-2\assets


---Starting trial: run-3
{'filter_size': 3, 'filter_num': 96}
Epoch 1/15
51/51 [==============================] - 2s 34ms/step
203/203 - 49s - loss: 0.0762 - accuracy: 0.9800 - val_loss: 0.0088 - val_accuracy: 0.9988 - 49s/epoch - 240ms/step
Epoch 2/15
51/51 [==============================] - 2s 32ms/step
203/203 - 48s - loss: 0.0100 - accuracy: 0.9988 - val_loss: 0.0036 - val_accuracy: 0.9994 - 48s/epoch - 236ms/step
Epoch 3/15
51/51 [==============================] - 2s 33ms/step
203/203 - 49s - loss: 0.0094 - accuracy: 0.9988 - val_loss: 0.0027 - val_accuracy: 0.9994 - 49s/epoch - 242ms/step
Epoch 4/15
51/51 [==============================] - 2s 31ms/step
203/203 - 49s - loss: 0.0093 - accuracy: 0.9983 - val_loss: 0.0341 - val_accuracy: 0.9907 - 49s/epoch - 240ms/step
Epoch 5/15
51/51 [==============================] - 2s 31ms/step
203/203 - 46s - loss: 0.0136 - accuracy: 0.9972 - val_loss: 0.0042 - val_accuracy: 0.9988 - 46s/epoch - 228ms/step
51/51 [==============================]

...

INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-10\assets


---Starting trial: run-11
{'filter_size': 7, 'filter_num': 96}
Epoch 1/15
51/51 [==============================] - 2s 33ms/step
203/203 - 46s - loss: 0.0738 - accuracy: 0.9801 - val_loss: 0.0112 - val_accuracy: 0.9988 - 46s/epoch - 226ms/step
Epoch 2/15
51/51 [==============================] - 2s 31ms/step
203/203 - 42s - loss: 0.0152 - accuracy: 0.9977 - val_loss: 0.0049 - val_accuracy: 0.9981 - 42s/epoch - 209ms/step
Epoch 3/15
51/51 [==============================] - 2s 33ms/step
203/203 - 43s - loss: 0.0102 - accuracy: 0.9987 - val_loss: 0.0032 - val_accuracy: 0.9988 - 43s/epoch - 209ms/step
Epoch 4/15
51/51 [==============================] - 3s 54ms/step
203/203 - 56s - loss: 0.0077 - accuracy: 0.9989 - val_loss: 0.0030 - val_accuracy: 0.9988 - 56s/epoch - 274ms/step
Epoch 5/15
51/51 [==============================] - 3s 52ms/step
203/203 - 72s - loss: 0.0059 - accuracy: 0.9991 - val_loss: 0.0279 - val_accuracy: 0.9957 - 72s/epoch - 354ms/step
Epoch 6/15
51/51 [===================

INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-11\assets


---Starting trial: run-12
{'filter_size': 7, 'filter_num': 128}
Epoch 1/15
51/51 [==============================] - 3s 52ms/step
203/203 - 73s - loss: 0.0771 - accuracy: 0.9759 - val_loss: 0.0051 - val_accuracy: 0.9988 - 73s/epoch - 357ms/step
Epoch 2/15
51/51 [==============================] - 3s 50ms/step
203/203 - 69s - loss: 0.0108 - accuracy: 0.9983 - val_loss: 0.0045 - val_accuracy: 0.9988 - 69s/epoch - 340ms/step
Epoch 3/15
51/51 [==============================] - 3s 49ms/step
203/203 - 70s - loss: 0.0101 - accuracy: 0.9987 - val_loss: 0.0066 - val_accuracy: 0.9988 - 70s/epoch - 344ms/step
Epoch 4/15
51/51 [==============================] - 3s 49ms/step
203/203 - 70s - loss: 0.0079 - accuracy: 0.9988 - val_loss: 0.0057 - val_accuracy: 0.9981 - 70s/epoch - 346ms/step
51/51 [==============================] - 3s 53ms/step - loss: 0.0045 - accuracy: 0.9988
INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-12\assets


INFO:tensorflow:Assets written to: saved_model/Model 1/fit/run-12\assets


In [7]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/hparam_tuning'


In [8]:
%load_ext tensorboard
%tensorboard --logdir 'Logs/Model 1/fit'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
